In [7]:
import numpy as np
import pandas as pd
import lightgbm

df = pd.DataFrame({
    "query_id":[i for i in range(100) for j in range(10)],
    "var1":np.random.random(size=(1000,)),
    "var2":np.random.random(size=(1000,)),
    "var3":np.random.random(size=(1000,)),
    "relevance":list(np.random.permutation([0,0,0,0,0, 0,0,0,1,1]))*100
})

In [27]:
df.query_id.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [8]:
train_df = df[:800]  # first 80%
validation_df = df[800:]  # remaining 20%

qids_train = train_df.groupby("query_id")["query_id"].count().to_numpy()
X_train = train_df.drop(["query_id", "relevance"], axis=1)
y_train = train_df["relevance"]

qids_validation = validation_df.groupby("query_id")["query_id"].count().to_numpy()
X_validation = validation_df.drop(["query_id", "relevance"], axis=1)
y_validation = validation_df["relevance"]

In [23]:
train_df.groupby("query_id")["query_id"].count().to_numpy()

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [21]:
validation_df.groupby("query_id")["query_id"].count().to_numpy()

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10])

In [9]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [28]:
X_train

,var1,var2,var3
0,0.045893,0.277622,0.454179
1,0.261332,0.360828,0.748708
2,0.262338,0.812328,0.249222
3,0.744301,0.638105,0.437958
4,0.893214,0.115536,0.714619
...,...,...,...
795,0.102366,0.159788,0.001801
796,0.092274,0.956503,0.802291
797,0.643740,0.492735,0.942712
798,0.597416,0.357232,0.670874


In [10]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10,
    verbose=10,
)

[10]	valid_0's ndcg@10: 0.510566
[20]	valid_0's ndcg@10: 0.519703
[30]	valid_0's ndcg@10: 0.52022
[40]	valid_0's ndcg@10: 0.532206
[50]	valid_0's ndcg@10: 0.543966
[60]	valid_0's ndcg@10: 0.546932
[70]	valid_0's ndcg@10: 0.548113
[80]	valid_0's ndcg@10: 0.538015
[90]	valid_0's ndcg@10: 0.525226
[100]	valid_0's ndcg@10: 0.520431


LGBMRanker(metric='ndcg', objective='lambdarank')

In [13]:
validation_df

,query_id,var1,var2,var3,relevance
800,80,0.852396,0.747550,0.857031,0
801,80,0.139155,0.015073,0.514989,0
802,80,0.490125,0.310547,0.472113,0
803,80,0.064505,0.913333,0.276656,0
804,80,0.204057,0.489041,0.451726,0
...,...,...,...,...,...
995,99,0.273644,0.506770,0.839744,0
996,99,0.562622,0.449007,0.512328,1
997,99,0.691616,0.249961,0.624470,0
998,99,0.604487,0.775340,0.457088,1


In [11]:
df

,query_id,var1,var2,var3,relevance
0,0,0.045893,0.277622,0.454179,0
1,0,0.261332,0.360828,0.748708,0
2,0,0.262338,0.812328,0.249222,0
3,0,0.744301,0.638105,0.437958,0
4,0,0.893214,0.115536,0.714619,0
...,...,...,...,...,...
995,99,0.273644,0.506770,0.839744,0
996,99,0.562622,0.449007,0.512328,1
997,99,0.691616,0.249961,0.624470,0
998,99,0.604487,0.775340,0.457088,1
